In [4]:
from sqlite3 import connect
import pandas as pd

conn = connect("machine2.db")

df = pd.read_sql_query("SELECT participantId, instanceId, participant_response, original_placement from social", conn)
df

,participantId,instanceId,participant_response,original_placement
0,97f4_7385706431,21,NaN,NaN
1,97f4_7385706431,57,NaN,NaN
2,97f4_7385706431,16,NaN,NaN
3,97f4_7385706431,72,NaN,NaN
4,97f4_7385706431,58,NaN,NaN
...,...,...,...,...
77162,zzru_7377077714,19,2.0,2.0
77163,zzru_7377077714,71,2.0,1.0
77164,zzru_7377077714,35,1.0,1.0
77165,zzru_7377077714,51,2.0,2.0


In [5]:
valid_df = df[df['participant_response'].isin([1,2]) & df['original_placement'].isin([1,2])].copy()
valid_df['group'] = valid_df['instanceId'] <= 36
valid_df['match'] = (valid_df['participant_response'] == valid_df['original_placement']).astype(int)
result = valid_df.groupby(['participantId', 'group'])['match'].mean().reset_index()

result.head()

,participantId,group,match
0,a265_7382527621,False,0.222222
1,a265_7382527621,True,0.833333
2,a295_7382607093,False,0.333333
3,a295_7382607228,False,0.666667
4,a295_7382607228,True,0.416667


In [7]:
def check_complete(series: pd.Series):
  mask = (series == 1) | (series == 2)
  n_values = mask.sum()
  if n_values == len(series):
    return 1
  if n_values == 0:
    return -1
  return 0

agg = result.pivot_table(index='participantId', columns='group', values='match')

complete = df.groupby('participantId')['participant_response'].apply(check_complete)
agg['complete'] = agg.index.map(complete.to_dict())
agg['mean'] = agg.index.map(valid_df.groupby('participantId')['match'].mean().to_dict())

agg.rename(columns={False: 'positive', True: 'negative'}, inplace=True)
agg.columns.name = None
agg.head()

,positive,negative,complete,mean
participantId,,,,
a265_7382527621,0.222222,0.833333,1,0.527778
a295_7382607093,0.333333,NaN,0,0.333333
a295_7382607228,0.666667,0.416667,1,0.541667
a3a7_7375337601,0.972222,0.388889,1,0.680556
a3kp_7384516585,0.138889,0.750000,1,0.444444


In [8]:
agg.to_csv("distance.csv")